# Use LD Proxy API to find the relevant positions for each of the PRS313 SNPs

In [3]:
import requests
import pandas as pd
import os
import io


# API key
api_key = "ac16be4ad92d"

# Base URL for LDProxy API
base_url = "https://ldlink.nih.gov/LDlinkRest/ldproxy"

population = "ALL"

window = 1000000

r2_threshold = 0.01

# Read the chromosome positions from the text file
PRS313_LD = pd.read_excel("../../Data/PRS313_with_23andMe.xlsx")


In [4]:

# Create a folder to store the CSV files
output_folder = "../ld_variants"
os.makedirs(output_folder, exist_ok=True)

# Iterate over each position
for sample in PRS313_LD:
    if sample["in_23andMe"] == True:
         continue

    # Extract chromosome and position from the line
    chrom = sample["Chromosome"]
    pos = sample["Positionb"]
    
    # Construct the API request URL
    url = f"{base_url}?var={chrom}:{pos}&pop={population}&r2_d=r2&window={window}&genome_build=grch37&token={api_key}"
    
    # Send the API request
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
            # Create a StringIO object from the data
            data_io = io.StringIO(response.text)

            # Read the data into a DataFrame using read_csv
            df = pd.read_csv(data_io, sep='\\t')
            
            # # Filter variants with high LD scores (e.g., R2 >= 0.8)
            # high_ld_variants = df[df["R2"].astype(float) >= 0.8]
            
            # Generate a unique filename for the CSV file
            output_file = os.path.join(output_folder, f"{chrom}_{pos}.csv")
            
            # Save the high LD variants to a CSV file
            df.to_csv(output_file, index=False)
            
            print(f"Saved high LD variants for {chrom}:{pos} to {output_file}")
    else:
        print(f"Failed to retrieve data for {chrom}:{pos}. Status code: {response.status_code}")

TypeError: string indices must be integers, not 'str'

# Find overlap b/w 23AndMe and LD


In [4]:
import os
import pandas as pd
import re

directory = "../../Data/ld_variants"

# Create a folder to store the CSV files
output_folder = "../../Data/ld_variants"
os.makedirs(output_folder, exist_ok=True)


def extract_coord_alleles(col_name):
    match = re.search(r'chr(\d+)_(\d+)_([ACGT]+)_([ACGT,]+)', col_name)
    if match:
        chr_num, position, ref_allele, alt_alleles = match.groups()
        return f'chr{chr_num}:{position}', f'({ref_allele}/{alt_alleles})'
    return None, None

# Create an empty list to store the matching variants across all chromosomes
matching_variants_all = []

training_data_folder = "../../Data/Raw_training_data_23andMe_union/"
folder = "../../Data/Filtered_unphased_training_data_union/"


# Iterate over each chromosome folder
for chrom_folder in os.listdir(output_folder):
    # Check if the folder is a chromosome folder
    if chrom_folder.startswith("chr"):
        # Extract the chromosome number
        chrom = chrom_folder[3:]

        files_folder = os.path.join(output_folder, chrom_folder)
        
        # Load the training data for the chromosome
        training_data = pd.read_parquet(f"{training_data_folder}23AndMe_PRS313_merged_chr{chrom}.parquet")
        
        # Create empty lists to store the matching columns and variants for the chromosome
        matching_columns = []
        matching_variants_chrom = []
        not_found_snps_chrom = []

        # Read all the files in the folder
        for filename in os.listdir(files_folder):

            # Check if the file is a CSV file
            if filename.endswith(".csv"):

                # Check if the position is an unknown position or a known position 
                position = filename.split('.')[0]

                # Find the column name with the position
                position_column = training_data.columns[training_data.columns.str.contains(position)]

                if (position_column[0].split("_")[-1] == "Known"):
                    # If the position is a known position, skip the file, because we don't need LD proxies for known positions
                    print(f"Skipping LD for {filename} because it is a known position. Column added to training data")

                    if position_column[0] not in matching_columns:

                        # Add the matching column to the list
                        matching_columns.append(position_column[0])

                    continue


                # Load the variants for the file
                variants = pd.read_csv(os.path.join(files_folder, filename))
                
                # Find the matching columns in the training data
                for column in training_data.columns:
                    coord, alleles = extract_coord_alleles(column)
                    if coord is not None and alleles is not None:
                        try:
                            # Got rid of allele matching, because of coding differences. GC--> C (in PRS313) is coded as (C/-) in dbSNP and LDProxy
                            mask = (variants['Coord'] == coord)
                            # mask = (variants['Coord'] == coord) & (variants['Alleles'] == alleles)
                                
                            if mask.any():
                                # Only append column if it has not been added before
                                if column not in matching_columns:

                                    # Check column added is an unknown position
                                    if (column.split("_")[-1] == "Unknown"):
                                        print(column)


                                    matching_columns.append(column)
                                    matching_variants_chrom.append(variants[mask])
                        except KeyError:
                            error_position = filename.split('.')[0]
                            not_found_snps_chrom.append(error_position)
                            print(f"SNP {error_position} not found in dbSNP and cannot be proxied using LDProxy")

                            # Find the columns in training_data.columns with positions within +/- 500K BP of the error_position
                            counter_error_added = 0
                            for col in training_data.columns:
                                coord, _ = extract_coord_alleles(col)
                                if coord is not None:
                                    col_position = int(coord.split(':')[1])
                                    error_bp = int(error_position.split('_')[1])
                                    if abs(col_position - error_bp) <= 500000 and col not in matching_columns:
                                        counter_error_added += 1
                                        matching_columns.append(col)

                            print(f"Added {counter_error_added} columns to matching_columns for missing data position: {error_position}")
                            
                            break

                print(f"Processed {filename}")
                print(f"Found {len(matching_columns)} matching columns")
                

        # Get the matching columns from the training data
        matching_data = training_data[matching_columns]

        os.makedirs(folder, exist_ok=True)
        save_path = f"{folder}/23AndMe_PRS313_merged_chr{chrom}_matching.parquet"
        # Save the matching data for the chromosome
        matching_data.to_parquet(save_path)

        # Concatenate the matching variants for the chromosome into a single dataframe
        matching_variants_chrom_df = pd.concat(matching_variants_chrom, ignore_index=True)
        
        # Append the matching variants for the chromosome to the overall list
        matching_variants_all.append(matching_variants_chrom_df)

        print(f"Saved to file {save_path}")
        print(f"Found {len(matching_columns)} matching columns")
        print(f"Found {len(matching_variants_chrom_df)} matching variants")

# Concatenate the matching variants from all chromosomes into a single dataframe
matching_variants_all_df = pd.concat(matching_variants_all, ignore_index=True)

# Save the dataframe with matching variants across all chromosomes
matching_variants_all_df.to_csv(f"{folder}/23AndMe_matching_variants.csv", index=False)

Skipping LD for 12_14413931.csv because it is a known position. Column added to training data
chr12_115796577_A_G_PRS313_Unknown
Processed 12_115796577.csv
Found 27 matching columns
Skipping LD for 12_115835836.csv because it is a known position. Column added to training data
Skipping LD for 12_120832146.csv because it is a known position. Column added to training data
chr12_103097887_C_T_PRS313_Unknown
Processed 12_103097887.csv
Found 60 matching columns
Skipping LD for 12_96027759.csv because it is a known position. Column added to training data


KeyboardInterrupt: 

# Save Reference and Alternate allele information to the file

In [5]:

matching_variants_all_df = pd.read_csv(f"{folder}/23AndMe_matching_variants.csv")

In [12]:
import os
import csv

# Directory containing the VCF files
vcf_directory = "../../Data/23AndMePositionsUnion"

# Read in the 23andMe_matching_variants.csv file
with open(f"{folder}/23andMe_matching_variants.csv", "r") as csv_file:
    reader = csv.DictReader(csv_file)
    data = list(reader)

# Process each row in the CSV
for row in data:
    # Extract the chromosome and position from the Coord column
    chrom, pos = row["Coord"].split(":")
    pos = pos.replace(",", "")  # Remove any commas from the position
    
    # Construct the path to the corresponding VCF file
    vcf_file = os.path.join(vcf_directory, chrom, f"{chrom}_pos{pos}.vcf")
    
    # Check if the VCF file exists
    if os.path.isfile(vcf_file):
        # Read the VCF file and extract the reference and alternate alleles
        with open(vcf_file, "r") as file:
            for line in file:
                if not line.startswith("#"):
                    fields = line.strip().split("\t")
                    ref = fields[3]
                    alt = fields[4].split(",")  # Split alternate alleles by comma
                    break
        
        # Update the row with the reference and alternate alleles
        row["Ref"] = ref
        row["Alt"] = ",".join(alt)  # Join alternate alleles with comma
    else:
        # If the VCF file doesn't exist, set the reference and alternate alleles to empty strings
        row["Ref"] = ""
        row["Alt"] = ""

# Write the updated data to a new CSV file
with open(f"{folder}/23andMe_matching_variants_updated.csv", "w", newline="") as csv_file:
    fieldnames = data[0].keys()
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(data)

# Create Final Training Data with +/- 500K BP Window

In [19]:
import os
import pandas as pd
import re

window_size = 250000
pattern = re.compile(r"chr\d+_(\d+)_")

# Create an empty list to store the matching variants across all chromosomes
matching_variants_all = []

for chrom in range(1, 23):
    # Load the training data for the chromosome
    training_data = pd.read_parquet(f"../../Data/Raw_training_data/23AndMe_PRS313_merged_chr{chrom}.parquet")
    
    # Get all columns with "PRS313" in the name
    prs313_unknown_columns = [col for col in training_data.columns if "PRS313_Unknown" in col]
    prs313_unknown_positions = [int(pattern.search(col).group(1)) for col in prs313_unknown_columns]
    prs313_unknown_positions_set = set(prs313_unknown_positions)
    
    # Get all columns in training_data that contain a number within +/- 500k of the PRS313_Unknown position
    filtered_columns = [col for col in training_data.columns if any(abs(int(pattern.search(col).group(1)) - pos) <= window_size for pos in prs313_unknown_positions_set)]
    
    training_data_filtered = training_data[filtered_columns]

    print(len(filtered_columns))

    # Save the filtered training data for the chromosome
    # training_data_filtered.to_parquet(f"../../Data/500k_window_filtered_data/23AndMe_PRS313_merged_chr{chrom}_filtered.parquet")

4138
3266
3217
2468
5178
3592
2290
2740
1953
2854
3720
2333
861
806
1252
3042
1565
1537
1847
1127
575
2007


In [28]:
chrom = 7

training_data_window = pd.read_parquet(f"../../Data/500k_window_filtered_data/23AndMe_PRS313_merged_chr{chrom}_filtered.parquet")
print(training_data_window.shape)

training_data_ld_proxy = pd.read_parquet(f"../../Data/filtered_training_data/23AndMe_PRS313_merged_chr{chrom}_matching.parquet")
print(training_data_ld_proxy.shape)

(2504, 2290)
(2504, 444)


In [17]:
training_data.columns

Index(['chr1_88177403_G_A', 'chr1_88127152_T_C', 'chr1_88208135_G_A',
       'chr1_88109828_G_A', 'chr1_88086894_C_A', 'chr1_88091734_C_T',
       'chr1_88116467_A_C', 'chr1_88220810_AC_A', 'chr1_88073752_G_T',
       'chr1_88120134_T_C',
       ...
       'chr1_172366806_A_G', 'chr1_172419651_T_G', 'chr1_172316842_G_A',
       'chr1_171934790_G_A', 'chr1_172632057_A_G', 'chr1_172627498_C_T',
       'chr1_172464519_T_G', 'chr1_172328767_T_TA_PRS313_Unknown',
       'chr1_121280485_A_G', 'chr1_121137155_A_G'],
      dtype='object', length=1132)

In [ ]:
'chr1_88177403_G_A', 'chr1_88127152_T_C', 'chr1_88208135_G_A',
       'chr1_88109828_G_A', 'chr1_88086894_C_A', 'chr1_88091734_C_T',
       'chr1_88116467_A_C', 'chr1_88220810_AC_A', 'chr1_88073752_G_T',
       'chr1_88120134_T_C',
       ...
       'chr1_172366806_